In [ ]:
import ROOT
import glob
import math
import pandas as pd, numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
path = "/home/berki/Software/TPCQCVis/data/userFiles/"
f = ROOT.TFile.Open(path+"clusterInvestigation.root","READ")
clus = f.Get("Clusters;1")

In [ ]:
# Extract info from all the MOs
df = pd.DataFrame()

for name in ["Q_Max","Q_Tot","Sigma_Time","Sigma_Pad","Time_Bin","N_Clusters"]:
    means_a=[]
    stdDevs_a=[]
    means_c=[]
    stdDevs_c=[]
    entries=[]
    for i in range(2018):
        c = clus.Get("c_Sides_"+name+";"+str(i+1))
        hist_a = c.GetPrimitive("c_Sides_"+name+"_3").GetPrimitive("h_Aside_1D_"+name)
        hist_a.GetXaxis().SetRange(2,hist_a.GetNbinsX())
        entries.append(hist_a.GetEntries())
        means_a.append(hist_a.GetMean())
        stdDevs_a.append(hist_a.GetStdDev())
        hist_c = c.GetPrimitive("c_Sides_"+name+"_4").GetPrimitive("h_Cside_1D_"+name)
        hist_c.GetXaxis().SetRange(2,hist_c.GetNbinsX())
        means_c.append(hist_c.GetMean())
        stdDevs_c.append(hist_c.GetStdDev())
    #print(means)
    df[name+'_entries'] = entries
    df[name+'_A'] = means_a
    df[name+'_C'] = means_c
    df[name] = (df[name+'_A']+df[name+'_C'])/2
    df[name+'_A_stdDev'] = stdDevs_a
    df[name+'_C_stdDev'] = stdDevs_c

In [ ]:
# Add runNumber metaData to values
df["runNumber"] = list(selected_files[selected_files["Name"]=="c_Sides_Q_Max"].RunNumber)

In [ ]:
# Aggregate the df to have one entry per runNumber
#aggregation_functions = {'price': 'sum', 'amount': 'sum', 'name': 'first'}
#df_new = df.groupby(df['id']).aggregate(aggregation_functions)
df_new = df.groupby(df['runNumber']).last()
df_new

In [ ]:
plt.rcParams["figure.figsize"] = (30,10)
ax = plt.gca()
df_new[df_new["Sigma_Time"]>0].reset_index().plot(x='runNumber',y='Sigma_Time',kind='scatter',ax=ax)

plt.show()

In [ ]:
selected_files

In [ ]:
len(set(list(selected_files[selected_files["Name"]=="c_Sides_Q_Max"].RunNumber)))